In [9]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [11]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv", encoding="latin-1")

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [12]:
from sklearn.model_selection import train_test_split

X = data["text"]
y = data["label"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(X_train.shape, X_val.shape)


(800,) (200,)


## Data Preprocessing

In [13]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [14]:
import re

def clean_html(text):
    text = re.sub(r'<script.*?>.*?</script>', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'<!--.*?-->', '', text)
    return text

X_train = X_train.apply(clean_html)
X_val = X_val.apply(clean_html)


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [15]:
def basic_clean(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.lower()

X_train = X_train.apply(basic_clean)
X_val = X_val.apply(basic_clean)


## Now let's work on removing stopwords
Remove the stopwords.

In [16]:
stop_words = set(stopwords.words("english"))

def remove_stop(text):
    return " ".join([w for w in text.split() if w not in stop_words])

X_train = X_train.apply(remove_stop)
X_val = X_val.apply(remove_stop)


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [17]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(w) for w in text.split()])

X_train = X_train.apply(lemmatize_text)
X_val = X_val.apply(lemmatize_text)


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

vectorizer = CountVectorizer(max_features=20)

bow = vectorizer.fit_transform(X_train)

words = vectorizer.get_feature_names_out()
counts = np.asarray(bow.sum(axis=0)).ravel()

print(list(zip(words, counts)))


[('account', np.int64(676)), ('also', np.int64(264)), ('bank', np.int64(623)), ('business', np.int64(439)), ('company', np.int64(359)), ('contact', np.int64(283)), ('country', np.int64(444)), ('dollar', np.int64(271)), ('foreign', np.int64(281)), ('fund', np.int64(601)), ('million', np.int64(371)), ('money', np.int64(762)), ('mr', np.int64(473)), ('name', np.int64(278)), ('nbsp', np.int64(387)), ('next', np.int64(291)), ('one', np.int64(297)), ('please', np.int64(313)), ('transaction', np.int64(432)), ('transfer', np.int64(319))]


## Extra features

In [20]:
import numpy as np

def extra_features(texts):
    money = texts.str.contains(r'\$|€|£').astype(int)
    suspicious = texts.str.contains(r'free|win|money|offer|bank|password|transfer').astype(int)
    length = texts.str.len()
    return np.vstack([money, suspicious, length]).T

extra_train = extra_features(X_train)
extra_val = extra_features(X_val)


## How would work the Bag of Words with Count Vectorizer concept?

In [21]:
count_vect = CountVectorizer(max_features=5000)

X_train_bow = count_vect.fit_transform(X_train)
X_val_bow = count_vect.transform(X_val)

print(X_train_bow.shape)


(800, 5000)


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)

X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)

print(X_train_tfidf.shape)


(800, 5000)


## And the Train a Classifier?

In [23]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

model = MultinomialNB()

model.fit(X_train_tfidf, y_train)

pred = model.predict(X_val_tfidf)

print("Accuracy:", accuracy_score(y_val, pred))


Accuracy: 0.985


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [24]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack
import numpy as np

def evaluate(Xtr, Xte, name):
    model = MultinomialNB()
    model.fit(Xtr, y_train)
    pred = model.predict(Xte)
    acc = accuracy_score(y_val, pred)
    print(f"{name} accuracy:", acc)


# ======================
# 1) Bag of Words only
# ======================
evaluate(X_train_bow, X_val_bow, "BOW")


# ======================
# 2) TF-IDF only
# ======================
evaluate(X_train_tfidf, X_val_tfidf, "TF-IDF")


# ======================
# 3) TF-IDF + extra features
# ======================
def extra_features(texts):
    money = texts.str.contains(r'\$|€|£').astype(int)
    suspicious = texts.str.contains(r'free|win|money|offer|bank|password|transfer').astype(int)
    length = texts.str.len()
    return np.vstack([money, suspicious, length]).T

extra_train = extra_features(X_train)
extra_val = extra_features(X_val)

X_train_final = hstack([X_train_tfidf, extra_train])
X_val_final = hstack([X_val_tfidf, extra_val])

evaluate(X_train_final, X_val_final, "TF-IDF + extra")


BOW accuracy: 0.98
TF-IDF accuracy: 0.985
TF-IDF + extra accuracy: 0.905
